<a href="https://colab.research.google.com/github/GabzBarbosa/Analise-NPS/blob/main/Analise_NPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
import os

os.makedirs("/content/nps/entrada", exist_ok=True)
os.makedirs("/content/nps/staging", exist_ok=True)
os.makedirs("/content/nps/saida", exist_ok=True)

print("Estrutura criada com sucesso")



Estrutura criada com sucesso


In [46]:
import pandas as pd

CAMINHO_XLSX = "/content/nps/entrada/nps.xlsx"
CAMINHO_CSV = "/content/nps/staging/nps.csv"

df_raw = pd.read_excel(CAMINHO_XLSX)
df_raw.columns = df_raw.columns.str.strip()

df_raw.to_csv(CAMINHO_CSV, index=False, encoding="utf-8-sig")

print("Arquivo XLSX convertido para CSV com sucesso")
print(f"Linhas: {len(df_raw)}")


Arquivo XLSX convertido para CSV com sucesso
Linhas: 939


In [47]:
df = pd.read_csv("/content/nps/staging/nps.csv")


In [48]:
COLUNAS_OBRIGATORIAS = [
    "Seller Id",
    "Seller Name",
    "Product Name",
    "Rating Value",
    "Rating Comment",
    "SUBMOTIVO_FINAL_V1"
]

faltando = [c for c in COLUNAS_OBRIGATORIAS if c not in df.columns]

if faltando:
    raise ValueError(f"Colunas obrigatórias ausentes: {faltando}")

print("Validação OK")


Validação OK


In [49]:
def classificar_nps(nota):
    if nota >= 9:
        return "Promotor"
    elif nota >= 7:
        return "Neutro"
    else:
        return "Detrator"

df["categoria_nps"] = df["Rating Value"].apply(classificar_nps)


In [50]:
def calcular_nps(grupo):
    total = len(grupo)
    promotores = (grupo["categoria_nps"] == "Promotor").sum()
    detratores = (grupo["categoria_nps"] == "Detrator").sum()
    return round(((promotores - detratores) / total) * 100, 2)


In [51]:
nps_seller = (
    df.groupby(["Seller Id", "Seller Name"])
    .apply(lambda x: pd.Series({
        "total_respostas": len(x),
        "promotores": (x["categoria_nps"] == "Promotor").sum(),
        "neutros": (x["categoria_nps"] == "Neutro").sum(),
        "detratores": (x["categoria_nps"] == "Detrator").sum(),
        "nps": calcular_nps(x)
    }))
    .reset_index()
)


/tmp/ipython-input-1543875417.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: pd.Series({


In [52]:
nps_produto = (
    df.groupby(["Seller Name", "Product Name"])
    .apply(lambda x: pd.Series({
        "total_respostas": len(x),
        "promotores": (x["categoria_nps"] == "Promotor").sum(),
        "neutros": (x["categoria_nps"] == "Neutro").sum(),
        "detratores": (x["categoria_nps"] == "Detrator").sum(),
        "nps": calcular_nps(x)
    }))
    .reset_index()
)


/tmp/ipython-input-2333567813.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: pd.Series({


In [53]:
def resumo_dores(grupo, limite=3):
    negativos = grupo[grupo["categoria_nps"] == "Detrator"]
    negativos = negativos.dropna(subset=["Rating Comment", "SUBMOTIVO_FINAL_V1"])

    textos = (
        negativos["SUBMOTIVO_FINAL_V1"].astype(str)
        + ": "
        + negativos["Rating Comment"].astype(str)
    )

    return " | ".join(textos.head(limite))

resumo_seller = (
    df.groupby("Seller Name")
    .apply(resumo_dores)
    .reset_index(name="principais_dores")
)

nps_seller = nps_seller.merge(resumo_seller, on="Seller Name", how="left")


/tmp/ipython-input-2936016381.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(resumo_dores)


In [54]:
# =========================
# 10. Top 3 tópicos por seller
# =========================
def top_3_topicos(grupo):
    # Considera apenas detratores
    negativos = grupo[grupo["categoria_nps"] == "Detrator"].copy()

    # Define campo de tópico (fallback)
    negativos["topico"] = negativos["SUBMOTIVO_FINAL_V1"].fillna(
        negativos["Rating Motives Negative Description"]
    )

    negativos = negativos.dropna(subset=["topico"])

    if negativos.empty:
        return pd.Series({
            "topico_1": None,
            "topico_2": None,
            "topico_3": None
        })

    topicos = (
        negativos["topico"]
        .value_counts()
        .head(3)
        .index
        .tolist()
    )

    # Garantir sempre 3 posições
    while len(topicos) < 3:
        topicos.append(None)

    return pd.Series({
        "topico_1": topicos[0],
        "topico_2": topicos[1],
        "topico_3": topicos[2]
    })

topicos_seller = (
    df.groupby("Seller Name")
    .apply(top_3_topicos)
    .reset_index()
)


/tmp/ipython-input-3469041522.py:42: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(top_3_topicos)


In [55]:
# =========================
# 11. Resumo executivo por seller
# =========================
def classificar_status(nps):
    if nps < 0:
        return "Crítico"
    elif nps < 30:
        return "Atenção"
    else:
        return "Saudável"

def gerar_resumo_executivo(row):
    status = classificar_status(row["nps"])

    topicos = [
        row.get("topico_1"),
        row.get("topico_2"),
        row.get("topico_3")
    ]
    topicos = [t for t in topicos if pd.notna(t)]

    resumo = (
        f"O seller apresenta status **{status}**, com NPS de {row['nps']} "
        f"em um total de {row['total_respostas']} avaliações. "
        f"Foram identificados {row['detratores']} detratores, indicando impacto "
        f"relevante na experiência do cliente."
    )

    if topicos:
        resumo += (
            f" Os principais temas recorrentes nos comentários negativos são: "
            f"{', '.join(topicos)}."
        )

    if pd.notna(row.get("principais_dores")):
        resumo += (
            f" Exemplos recorrentes de reclamações incluem: "
            f"{row['principais_dores']}."
        )

    resumo += (
        " Recomenda-se análise prioritária das causas e definição de plano de ação "
        "para mitigação dos pontos críticos identificados."
    )

    return resumo

nps_seller["status_executivo"] = nps_seller["nps"].apply(classificar_status)
nps_seller["resumo_executivo"] = nps_seller.apply(gerar_resumo_executivo, axis=1)


In [56]:
nps_seller = nps_seller.sort_values(["nps", "detratores"], ascending=[True, False])
nps_produto = nps_produto.sort_values(["nps", "detratores"], ascending=[True, False])


In [57]:
nps_seller = nps_seller.merge(
    topicos_seller,
    on="Seller Name",
    how="left"
)


In [58]:
SAIDA = "/content/nps/saida"

nps_seller.to_csv(f"{SAIDA}/nps_por_seller.csv", index=False)
nps_produto.to_csv(f"{SAIDA}/nps_por_produto.csv", index=False)

print("Processo concluído com sucesso")


Processo concluído com sucesso
